The goal here is to evaluate the errors introduced adversarial training by a analyzing the added noise

In [147]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import kurtosis, moment

In [148]:
origWeights = np.load('origWeights.npy', allow_pickle=True)
modifiedWeights = np.load('modifiedWeights.npy', allow_pickle=True)
origBiases = np.load('origBiases.npy', allow_pickle=True)
modifiedBiases = np.load('modifiedBiases.npy', allow_pickle=True)

In [149]:
def flatten_listArrays(listArrays):
    flattenedArray = np.array([])
    for array in listArrays:
        flattenedArray = np.append(flattenedArray, array.flatten())
    return flattenedArray

In [150]:
def compute_minAndMax(listArrays):
    '''
    Function to find the smallest and largest elements 
    in a list of numpy arrays
    '''
    minVal = np.inf
    maxVal = -np.inf
    for arr in listArrays:
        if arr.min() < minVal:
            minVal = arr.min()
        if arr.max() > maxVal:
            maxVal = arr.max()
    return minVal, maxVal

In [151]:
def measure_noise(origValues, modifiedValues):
    noiseValues = []
    for origArr, modifiedArr in zip(origValues, modifiedValues):
        # error_arr is the element-wise difference of all values in the array
        errorArr = origArr - modifiedArr
        noiseValues.append(errorArr)
    return noiseValues

In [152]:
# The CNN for MNIST and the CNN for CIFAR-10, both have convolutional layers followed by a dense layer and then
# a softmax layer. So, the number of conv layers is 2 less than the total number of layers.
numConvLayers = len(origWeights) - 2
modelNoise = measure_noise(origWeights, modifiedWeights)
modelNoiseConvLayers = modelNoise[:numConvLayers]

In [153]:
for layer in modelNoiseConvLayers:
    print(layer.T.shape)
colors = []
if numConvLayers == 3:
    palette = ['r', 'g', 'b']
else:
    palette = ['r', 'g', 'b', 'y', 'c']

# Create a array of colors representing the layers to which the various 9*1 vectors belong to.
for layer,c in zip(modelNoiseConvLayers, palette):
    for kernel in layer.T:
        for matrix in kernel:
            colors.append(c)

(32, 1, 3, 3)
(64, 32, 3, 3)
(64, 64, 3, 3)


In [154]:
modelNoise1D = flatten_listArrays(modelNoise)
print(np.std(modelNoise1D))

0.00038695610439112816


In [155]:
def compute_kurtosis(data):
    return moment(data, 4)/(moment(data, 2) ** 2)

In [156]:
# Generate the Gaussian noise for the same number of parameters as contained in the model.
sigmaArray = np.random.uniform(1e-5, 8e-3, 20)
sigmaArray.sort()
for sigma in sigmaArray:
    numParams = len(flatten_listArrays(origWeights))
    randNoise = np.random.normal(0, sigma, numParams)
    b2 = compute_kurtosis(randNoise)
    result = 'Negative'
    if b2 <= 2.95 or b2 >= 3.05:
        result = 'Positive'
    print('{0:.3g} & {1:.2f} & {2} \\\\'.format(sigma**2, b2, result))

6.9e-08 & 3.00 & Negative \\
2.63e-07 & 3.00 & Negative \\
5.93e-07 & 2.99 & Negative \\
4.81e-06 & 3.01 & Negative \\
7.16e-06 & 3.01 & Negative \\
8.58e-06 & 3.00 & Negative \\
8.75e-06 & 3.00 & Negative \\
1.03e-05 & 2.99 & Negative \\
1.2e-05 & 3.01 & Negative \\
1.28e-05 & 2.98 & Negative \\
1.3e-05 & 3.00 & Negative \\
1.43e-05 & 3.01 & Negative \\
1.56e-05 & 3.00 & Negative \\
2.01e-05 & 3.00 & Negative \\
2.54e-05 & 3.00 & Negative \\
3.68e-05 & 2.99 & Negative \\
4.37e-05 & 2.99 & Negative \\
4.56e-05 & 2.98 & Negative \\
5.92e-05 & 3.00 & Negative \\
5.98e-05 & 3.02 & Negative \\


In [157]:
# Compute random noise for all of the layers in the same shape as the neural network weights.
def compute_realNoise(modelNoise, sigma):
    """ Generates real noise consisting of model noise with varying amounts of random noise. The generated noise 
    has the same shape as the input model noise. Both modelNoise and randNoise are expected to have the same 
    length """
    realNoise = []
    randNoiseArray = []
    for modelLayer in modelNoise:
        randLayer = np.random.normal(0,sigma, modelLayer.shape)
        realLayer = modelLayer + randLayer
        realNoise.append(realLayer)
        #randNoiseArray.append(randLayer)
    #variance = np.var(flatten_listArrays(randNoiseArray))
    #print(sigma ** 2, variance)
    return realNoise

In [158]:
v = np.var(flatten_listArrays(modelNoise))
print('variance of model noise : {0:.3g}'.format(v))
for sigma in sigmaArray:
    realNoise = compute_realNoise(modelNoise, sigma)
    realNoise1D = flatten_listArrays(realNoise)
    b2 = compute_kurtosis(realNoise1D)
    result = 'Negative'
    if b2 <= 2.95 or b2 >= 3.05:
        result = 'Positive'
    print('{0:.3g} & {1:.2f} & {2} \\\\'.format(sigma**2, b2, result))

variance of model noise : 1.5e-07
6.9e-08 & 18.32 & Positive \\
2.63e-07 & 7.32 & Positive \\
5.93e-07 & 4.29 & Positive \\
4.81e-06 & 3.02 & Negative \\
7.16e-06 & 3.03 & Negative \\
8.58e-06 & 2.99 & Negative \\
8.75e-06 & 2.99 & Negative \\
1.03e-05 & 2.98 & Negative \\
1.2e-05 & 3.01 & Negative \\
1.28e-05 & 3.03 & Negative \\
1.3e-05 & 3.02 & Negative \\
1.43e-05 & 3.00 & Negative \\
1.56e-05 & 3.00 & Negative \\
2.01e-05 & 2.99 & Negative \\
2.54e-05 & 3.02 & Negative \\
3.68e-05 & 2.98 & Negative \\
4.37e-05 & 3.01 & Negative \\
4.56e-05 & 2.99 & Negative \\
5.92e-05 & 3.01 & Negative \\
5.98e-05 & 2.99 & Negative \\
